In [ ]:
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
import torch, torchvision
torch.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-1nudkosh
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-1nudkosh
     |████████████████████████████████| 596 kB 7.8 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5-py3-none-any.whl size=65351 sha256=827631026aacd12c3e62a60fa142b568d9a1a250d39d32a6547de5fb4aceec05
  Stored in directory: /tmp/pip-ephem-wheel-cache-ds4i1f2a/wheels/24/1d/09/8167de727fe5b74f832b6fcb5d9069d8f03ca29f337bfe484d
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31549 sha256=81dd865460538203d8ccc72d889277f7b95bfa340e294c6209f6ee1af9663b5e
  Stored in directory: /root/.cache/pip/wheels/aa/cc/ed/ca4e88beef65

'1.12.0+cu113'

In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

Cloning into 'detectron2_repo'...
remote: Enumerating objects: 14465, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 14465 (delta 14), reused 26 (delta 12), pack-reused 14425
Receiving objects: 100% (14465/14465), 5.93 MiB | 24.47 MiB/s, done.
Resolving deltas: 100% (10455/10455), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/detectron2_repo
     |████████████████████████████████| 79 kB 1.6 MB/s 
     |████████████████████████████████| 151 kB 16.5 MB/s 
     |████████████████████████████████| 1.4 MB 46.6 MB/s 
     |████████████████████████████████| 512 kB 46.5 MB/s 
     |████████████████████████████████| 248 kB 46.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 96 kB 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Model Part**

In [ ]:
import detectron2
from PIL import Image
import cv2
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
import os
from detectron2.utils.visualizer import Visualizer
import numpy as np

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("deepfashion_validation",)
cfg.DATASETS.TEST = ()
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 1500
cfg.SOLVER.STEPS = (1000, )
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13
cfg.TEST.EVAL_PERIOD = 500
cfg.OUTPUT_DIR = './drive/MyDrive/detectron/data/DeepFashion2/'

In [ ]:
cfg.MODEL.DEVICE = "cpu"

In [ ]:
# Need to upload model(model_path) and testing image(original_img_path) in google drive 
model_path = "/content/drive/MyDrive/Colab Notebooks/AI-ICT Project/model_final.pth"
original_img_path = "/content/drive/MyDrive/Colab Notebooks/AI-ICT Project/LV3.jpeg"

# Define output path
result_img_path = "/content/drive/MyDrive/Colab Notebooks/AI-ICT Project/result_img.jpg"
crop_result_img_path = "/content/drive/MyDrive/Colab Notebooks/AI-ICT Project/crop_result_img.jpg"

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, model_path)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.55   # testing threshold -> 0.7 ?
#cfg.DATASETS.TEST = ("/content/drive/MyDrive/detectron/data/validation/image", )

In [ ]:
predictor = DefaultPredictor(cfg)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


In [ ]:
# predictor 정의  
predictor = detectron2.engine.defaults.DefaultPredictor(cfg)
img = cv2.imread(original_img_path)
outputs = predictor(img)
instances = outputs["instances"].to('cpu')


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
# 테스트 이미지 넣어서 인식 결과 표현된 이미지 출력 후 저장 (카테고리명, 예측확률, instance segmentation 결과 색칠, 바운딩박스 표현됨)
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(instances)
cv2.imwrite(result_img_path, v.get_image()[:, :, ::-1])

True

In [ ]:
print(instances)

Instances(num_instances=1, image_height=807, image_width=799, fields=[pred_boxes: Boxes(tensor([[307.5757, 132.5234, 521.0787, 585.5564]])), scores: tensor([0.7049]), pred_classes: tensor([11]), pred_masks: tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]]])])


In [ ]:

# 인식된 객체 카테고리명 추출, 바운딩박스 좌표대로 이미지 크롭
pred_classes = instances.pred_classes
boxes = instances.pred_boxes

if isinstance(boxes, detectron2.structures.boxes.Boxes):
    boxes = boxes.tensor.numpy()
else:
    boxes = np.asarray(boxes)

img_original = Image.open(original_img_path)
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
#metadata = MetadataCatalog.get(img_original)
#class_catalog = metadata.things_classes

for idx, coordinates in enumerate(boxes):
    class_index = pred_classes[idx]
    #class_name = class_catalog[class_index] # 인식된 객체 카테고리명 ex.'vest_dress'

    box = boxes[idx] # pred_boxes
    x_top_left = box[0]
    y_top_left = box[1]
    x_bottom_right = box[2]
    y_bottom_right = box[3]

    crop_img = img_original.crop((int(x_top_left), int(y_top_left), int(x_bottom_right), int(y_bottom_right)))
    crop_img.save(crop_result_img_path, "JPEG")

In [ ]:
boxes

array([[307.5757 , 132.52338, 521.07874, 585.5564 ]], dtype=float32)

In [ ]:
boxes[0]

array([307.5757 , 132.52338, 521.07874, 585.5564 ], dtype=float32)

In [ ]:
#own_box = np.array([207.5757 , 32.52338, 421.07874, 485.5564 ], dtype=np.float32)


# **HTML & Search API Part**

In [ ]:
# path for HTML beatiful soup
# In PC, need original image (ori_desktop_img_path) and output image (desktop_result_img_path)  from detectron model
desktop_result_img_path = "G:/My Drive/Colab Notebooks/AI-ICT Project/result_img.jpg"
ori_desktop_img_path = "G:/My Drive/Colab Notebooks/AI-ICT Project/LV3.JPEG"

# In google drive, need initial HTML file (initial_html_path)
initial_html_path = "/content/drive/MyDrive/Colab Notebooks/AI-ICT Project/initial_html.html"
# FInal html (modified_html_path) will be generated in google drive
modified_html_path = "/content/drive/MyDrive/Colab Notebooks/AI-ICT Project/modified_html.html"

In [ ]:
# Google Search API used

import requests
import webbrowser
import re

#filePath = '/content/drive/MyDrive/Colab Notebooks/AI-ICT Project/LV3.jpeg'
filePath = crop_result_img_path
searchUrl = 'http://www.google.hr/searchbyimage/upload'
multipart = {'encoded_image': (filePath, open(filePath, 'rb')), 'image_content': ''}

response = requests.post(searchUrl, files=multipart, allow_redirects=False)
# fetchUrl = response.headers['Location']
# webbrowser.open(fetchUrl)

y = response.content
z = str(y)


regularex =  regularex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|(([^\s()<>]+|(([^\s()<>]+)))))+(?:(([^\s()<>]+|(([^\s()<>]+))))|[^\s`!()[]{};:'\".,<>?«»“”‘’]))"
urlsrc = re.findall(regularex,z)
print(urlsrc[0][0])

generated_link = urlsrc[0][0][:-1]
print(generated_link)

http://www.google.hr/search?tbs=sbi:AMhZZis9xKYmFgT71vxhD4ny_1KzcUBcV8DafzFw3-bnEessRgbQuSoJ9Rw9N1fKT_1s1wpC0nISrDBaUW_1G_1_1D9GnDBgqngdJ8moQ7FbI-bUo5OUBTPIirrO7AlyT3cu8Eqx55zMedQtsskUt7CJ1kbEivvgBesSorrDjCl7DoHqicqQHCX00JZtpsNUrIwZro_1ZJOgLZvp6DQDnOo72yfZH_195-t7G4O_133WRLJYdeLfDPDERddQJ3g-DsnD_1jngszyCwNjpGAKWbKpjkED6Cf7Seo0mDQ-jChsZEs-jqMvm9uqLZoF8IbBhljjVigpT2dN2Mtt8NILkMfr-uTbc3q4WWEEJBrfDuQ"
http://www.google.hr/search?tbs=sbi:AMhZZis9xKYmFgT71vxhD4ny_1KzcUBcV8DafzFw3-bnEessRgbQuSoJ9Rw9N1fKT_1s1wpC0nISrDBaUW_1G_1_1D9GnDBgqngdJ8moQ7FbI-bUo5OUBTPIirrO7AlyT3cu8Eqx55zMedQtsskUt7CJ1kbEivvgBesSorrDjCl7DoHqicqQHCX00JZtpsNUrIwZro_1ZJOgLZvp6DQDnOo72yfZH_195-t7G4O_133WRLJYdeLfDPDERddQJ3g-DsnD_1jngszyCwNjpGAKWbKpjkED6Cf7Seo0mDQ-jChsZEs-jqMvm9uqLZoF8IbBhljjVigpT2dN2Mtt8NILkMfr-uTbc3q4WWEEJBrfDuQ


In [ ]:
# Part to get website URL list
from bs4 import BeautifulSoup
import requests


url = generated_link

headers = {
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'accept-language' : 'en-KR',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
}

all_links=[]
soup = BeautifulSoup(requests.get(url, headers=headers).text, 'lxml')
#print(soup)
for href in soup.find('div', {'id': 'search'}).find_all('a'):
    # only take amazon shopping website
    if href.get('href').startswith('https://www.amazon'):
        all_links.append(href.get('href'))

search_api_URL_list = list(set(all_links))
print(search_api_URL_list)

['https://www.amazon.com/Dresses-Casual-Pockets-Pleated-Wedding/dp/B09HX82PJ2', 'https://www.amazon.co.uk/Playshoes-Girls-Protection-Bathing-Swimsuit/dp/B00G5EAAQY', 'https://www.amazon.com/ENKA-Womens-Stretchy-Palazzo-Pockets/dp/B07Y1WQLDY', 'https://www.amazon.com/Playshoes-Girls-Collection-Bathing-Swimsuit/dp/B00X6DBWVY', 'https://www.amazon.com/polka-dot-swimsuits-women/s?k=polka+dot+swimsuits+for+women', 'https://www.amazon.com/Uniarmoire-Womens-Polka-Swimdress-Swimsuit/dp/B07KZYZWMG', 'https://www.amazon.com/Septangle-Womens-Shaping-Ruffled-Swimdress/dp/B08PVKDJBZ']


In [ ]:
# Part to modify the initial html
import bs4

# load the file
with open(initial_html_path) as inf:
    file1 = inf.read()
    soup = bs4.BeautifulSoup(file1)

# create new box
image = soup.new_tag("img", src= ori_desktop_img_path, height="750", width="500", usemap = "#imagemap")
image2 = soup.new_tag("img", src= desktop_result_img_path, height="750", width="500", usemap = "#imagemap")

new_box = soup.new_tag("area", coords= boxes[0], href=  search_api_URL_list[1], target="blank")


# insert it into the document
soup.h1.append(image)
soup.h2.append(image2)
soup.map.append(new_box)


# save the file again
with open(modified_html_path, "w") as outf:
    outf.write(str(soup))

print("Done!")

Done!
